In [2]:
import pandas as pd
import numpy as np
import h5torch
import py2bit

In [3]:
peaks = pd.read_csv("/data/home/natant/Negatives/Data/Encode690/ENCODE_hg38_subset_101bp_celltypes/A549_subset.bed", sep="\t", header=None, names=["chr", "start", "end", "TF"])

In [4]:
# Get unique TFs
unique_tfs = peaks["TF"].unique()
tf_to_index = {tf: idx for idx, tf in enumerate(unique_tfs)}

# Initialize a binary presence array
num_peaks = len(peaks)
num_tfs = len(unique_tfs)
tf_presence = np.zeros((num_peaks, num_tfs), dtype=int)

In [5]:
# Iterate through each peak and mark its presence in the array
for peak_index, peak in peaks.iterrows():
    tf_index = tf_to_index[peak["TF"]]
    tf_presence[peak_index, tf_index] = 1

    # Handle overlapping peaks
    overlapping_peaks = peaks[
        (peaks["chr"] == peak["chr"]) &
        (peaks["start"] < peak["end"]) &
        (peaks["end"] > peak["start"]) &
        (peaks.index != peak_index)
    ]
    for _, overlap_peak in overlapping_peaks.iterrows():
        overlap_tf_index = tf_to_index[overlap_peak["TF"]]
        tf_presence[peak_index, overlap_tf_index] = 2

In [6]:
(tf_presence==2).sum()

625

In [3]:
genome_path = "/data/home/natant/plmBind/Data/hg38.2bit"
tb = py2bit.open(genome_path)
mapping = {"A": 0, "T": 1, "C": 2, "G": 3, "N": 4}

chr_allow_list = list(np.arange(1, 23).astype(str)) + ["X", "Y"]
chr_allow_list = ["chr" + c for c in chr_allow_list]
chr_per_peaks = []
pos_per_peaks = []
len_per_peaks = []

def create_tf_presence_dataset(bed_file, output_h5t_file):
    # Read the BED file
    peaks = pd.read_csv(bed_file, sep="\t", header=None, names=["chr", "start", "end", "TF"])
    
    # Get unique TFs
    unique_tfs = peaks["TF"].unique()
    tf_to_index = {tf: idx for idx, tf in enumerate(unique_tfs)}
    
    # Initialize a binary presence array
    num_peaks = len(peaks)
    num_tfs = len(unique_tfs)
    tf_presence = np.zeros((num_tfs, 1001), dtype=int) #! change back to num_peaks

    # Iterate through each peak and mark its presence in the array
    chr_old = ""
    
    for peak_index, peak in peaks.iterrows():
        if peak_index > 1000:
            break

        if peak["chr"] not in chr_allow_list:
            continue
        chr_now = peak["chr"]
        if chr_now != chr_old:
            print(chr_now)
            chr_old = chr_now
        if peak_index % 1000 == 0:
            print(peak_index)

        tf_index = tf_to_index[peak["TF"]]
        tf_presence[tf_index, peak_index] = 1

        # Handle overlapping peaks
        overlapping_peaks = peaks[
            (peaks["chr"] == peak["chr"]) &
            (peaks["start"] < peak["end"]) &
            (peaks["end"] > peak["start"]) &
            (peaks.index != peak_index)
        ]
        for _, overlap_peak in overlapping_peaks.iterrows():
            overlap_tf_index = tf_to_index[overlap_peak["TF"]]
            tf_presence[overlap_tf_index, peak_index] = 2

        # register peak location
        middle = int((int(peak["start"]) + int(peak["end"])) / 2)
        chr_per_peaks.append(peak["chr"])
        pos_per_peaks.append(middle)
        len_per_peaks.append(int(peak["end"]) - int(peak["start"]))

    # Gather protein names in order they were filled in
    index_to_TF = {v : k for k, v in tf_to_index.items()}
    prot_names = np.array([index_to_TF[i] for i in range(len(index_to_TF))])

    # Gather sequence in int8 format:
    genome = {}
    for chr_ in chr_allow_list:
        genome[chr_] = np.array([mapping[bp] for bp in tb.sequence(chr_)], dtype="int8")

    # # Save the dataset to an HDF5 file
    # f = h5torch.File(output_h5t_file, "w")
    # f.register(
    #     tf_presence, 
    #     "central", 
    #     mode="N-D", 
    #     dtype_save="int", 
    #     dtype_load="int"
    # )

    # f.register(
    #     prot_names.astype(bytes),
    #     axis=0,
    #     name = "prot_names",
    #     mode = "N-D",
    #     dtype_save = "bytes",
    #     dtype_load = "str"
    # )
    # f.register(
    #     np.array(chr_per_peaks).astype(bytes),
    #     axis=1,
    #     name="peak_ix_to_chr",
    #     mode = "N-D",
    #     dtype_save="bytes",
    #     dtype_load="str"
    # )

    # f.register(
    #     np.array(pos_per_peaks),
    #     axis=1,
    #     name="peak_ix_to_pos",
    #     mode = "N-D",
    #     dtype_save="int64",
    #     dtype_load="int64"
    # )

    # f.register(
    #     np.array(len_per_peaks),
    #     axis=1,
    #     name="peak_ix_to_len",
    #     mode = "N-D",
    #     dtype_save="int64",
    #     dtype_load="int64"
    # )

    # for k, v in genome.items():
    #     f.register(
    #         v,
    #         axis="unstructured",
    #         name=k,
    #         mode="N-D",
    #         dtype_save="int8",
    #         dtype_load="int8",
    #     )

    # f.close()

In [23]:
# Example usage
create_tf_presence_dataset("/data/home/natant/Negatives/Data/Encode690/ENCODE_hg38_subset_101bp_celltypes_ATAC/A549_chip_concatenated_merged.bed", "/data/home/natant/Negatives/testing_ground/20250401_test.h5t")

chr1
0
1000


In [4]:
chr_per_peaks = []
pos_per_peaks = []
len_per_peaks = []

chr_allow_list = list(np.arange(1, 23).astype(str)) + ["X", "Y"]
chr_allow_list = list(np.arange(1, 4).astype(str)) + ["X"] #! FOR DEBUGGING
chr_allow_list = ["chr" + c for c in chr_allow_list]

peaks = pd.read_csv("/data/home/natant/Negatives/Data/Encode690/ENCODE_hg38_subset_101bp_celltypes_ATAC/A549_chip_concatenated_merged.bed", sep="\t", header=None, names=["chr", "start", "end", "TF"])

# Get unique TFs
unique_tfs = peaks["TF"].unique()
tf_to_index = {tf: idx for idx, tf in enumerate(unique_tfs)}

# Initialize a binary presence array
filtered_peaks = peaks[peaks["chr"].isin(chr_allow_list)]
filtered_peaks = filtered_peaks.reset_index(drop=True)
num_peaks = len(filtered_peaks)
num_tfs = len(unique_tfs)
tf_presence = np.zeros((num_tfs, num_peaks), dtype=int)

# Iterate through each peak and mark its presence in the array
chr_old = ""

for peak_index, peak in filtered_peaks.iterrows():
    # if peak["chr"] not in chr_allow_list:
    #     continue
    chr_now = peak["chr"]
    if chr_now != chr_old:
        print(chr_now)
        chr_old = chr_now
    if peak_index % 1000 == 0:
        print(peak_index)

    tf_index = tf_to_index[peak["TF"]]
    tf_presence[tf_index, peak_index] = 1

    # Handle overlapping peaks
    overlapping_peaks = filtered_peaks[
        (filtered_peaks["chr"] == peak["chr"]) &
        (filtered_peaks["start"] < peak["end"]) &
        (filtered_peaks["end"] > peak["start"]) &
        (filtered_peaks.index != peak_index)
    ]
    for _, overlap_peak in overlapping_peaks.iterrows():
        overlap_tf_index = tf_to_index[overlap_peak["TF"]]
        tf_presence[overlap_tf_index, peak_index] = 2

    # register peak location
    middle = int((int(peak["start"]) + int(peak["end"])) / 2)
    chr_per_peaks.append(peak["chr"])
    pos_per_peaks.append(middle)
    len_per_peaks.append(int(peak["end"]) - int(peak["start"]))

# Gather protein names in order they were filled in
index_to_TF = {v : k for k, v in tf_to_index.items()}
prot_names = np.array([index_to_TF[i] for i in range(len(index_to_TF))])

# Gather sequence in int8 format:
genome = {}
for chr_ in chr_allow_list:
    genome[chr_] = np.array([mapping[bp] for bp in tb.sequence(chr_)], dtype="int8")

# # Save the dataset to an HDF5 file
# f = h5torch.File("/data/home/natant/Negatives/testing_ground/20250402_test_longer.h5t", "w")
# f.register(
#     tf_presence, 
#     "central", 
#     mode="N-D", 
#     dtype_save="int", 
#     dtype_load="int"
# )

# f.register(
#     prot_names.astype(bytes),
#     axis=0,
#     name = "prot_names",
#     mode = "N-D",
#     dtype_save = "bytes",
#     dtype_load = "str"
# )
# f.register(
#     np.array(chr_per_peaks).astype(bytes),
#     axis=1,
#     name="peak_ix_to_chr",
#     mode = "N-D",
#     dtype_save="bytes",
#     dtype_load="str"
# )

# f.register(
#     np.array(pos_per_peaks),
#     axis=1,
#     name="peak_ix_to_pos",
#     mode = "N-D",
#     dtype_save="int64",
#     dtype_load="int64"
# )

# f.register(
#     np.array(len_per_peaks),
#     axis=1,
#     name="peak_ix_to_len",
#     mode = "N-D",
#     dtype_save="int64",
#     dtype_load="int64"
# )

# for k, v in genome.items():
#     f.register(
#         v,
#         axis="unstructured",
#         name=k,
#         mode="N-D",
#         dtype_save="int8",
#         dtype_load="int8",
#     )

# f.close()

chr1
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000


KeyboardInterrupt: 

In [5]:
len_per_peaks

[106,
 40,
 120,
 42,
 40,
 40,
 169,
 182,
 101,
 101,
 101,
 145,
 490,
 101,
 101,
 79,
 40,
 75,
 114,
 589,
 101,
 65,
 259,
 101,
 90,
 66,
 98,
 40,
 47,
 118,
 263,
 101,
 101,
 135,
 60,
 40,
 289,
 74,
 230,
 75,
 145,
 180,
 101,
 95,
 45,
 63,
 233,
 101,
 101,
 101,
 112,
 160,
 101,
 269,
 177,
 534,
 101,
 101,
 101,
 101,
 114,
 151,
 76,
 115,
 77,
 279,
 101,
 74,
 101,
 343,
 101,
 145,
 101,
 46,
 41,
 184,
 101,
 70,
 101,
 301,
 101,
 101,
 43,
 109,
 58,
 64,
 225,
 101,
 72,
 722,
 194,
 104,
 45,
 844,
 101,
 82,
 152,
 101,
 69,
 61,
 62,
 75,
 54,
 85,
 41,
 328,
 157,
 213,
 101,
 101,
 101,
 101,
 101,
 101,
 101,
 50,
 274,
 101,
 54,
 40,
 40,
 234,
 103,
 64,
 120,
 101,
 41,
 101,
 387,
 101,
 86,
 55,
 184,
 311,
 101,
 101,
 161,
 44,
 63,
 138,
 123,
 53,
 91,
 131,
 78,
 70,
 51,
 212,
 101,
 47,
 40,
 46,
 377,
 40,
 102,
 131,
 154,
 134,
 311,
 101,
 199,
 57,
 77,
 172,
 101,
 298,
 101,
 50,
 517,
 52,
 82,
 249,
 75,
 89,
 77,
 78,
 994,
 101,

In [8]:
len(chr_per_peaks)

70159

In [9]:
len(pos_per_peaks)

70159

In [ ]:
prot_names

array(['ATAC_peak', 'CTCF', 'YY1_(SC-281)', 'CREB1_(SC-240)', 'Max',
       'TCF12', 'FOSL2', 'ELF1_(SC-631)', 'BHLHE40', 'ATF3', 'USF-1',
       'ETS1', 'SIX5', 'ZBTB33', 'FOXA1_(SC-101058)'], dtype='<U17')

In [24]:
# Open the H5torch file in read mode
file_path = "/data/home/natant/Negatives/testing_ground/20250320_test.h5t"
f =  h5torch.File(file_path, "r")
# Access a specific dataset (e.g., "central")
central_data = f["central"][:]
print("Shape of 'central' dataset:", central_data.shape)

Shape of 'central' dataset: (15, 1001)


In [27]:
f["1"].keys()

<KeysViewHDF5 ['peak_ix_to_chr', 'peak_ix_to_len', 'peak_ix_to_pos']>

In [30]:
sum(f["1/peak_ix_to_len"][:]==101)/f["1/peak_ix_to_len"][:].shape[0]

0.25674325674325676

In [32]:
f["0/prot_names"][:]==b"ATAC_peak"

array([ True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False])

In [37]:
f["central"][:].shape

(15, 1001)

In [ ]:
sum(f["central"][0,:]==2) #! So 243 overlaps (on 1001 peaks)

243

In [ ]:
sum(f["central"][0,:]==0) #! 11 times that the atac peak is not present for a chip peak (on 1001 peaks)

11

In [ ]:
sum(f["central"][0,:]==1)/f["central"][0,:].shape[0] #! around 75% of the atac peak has no corresponding chip peak (on 1001 peaks) 
#! THIS MAKES SENSE BUT HOW DO DEAL WITH THE OVERLAP?

0.7462537462537463

In [49]:
(f["central"][1:,:]==1).sum(axis=1)

array([85, 16, 43, 18, 11, 16, 12,  7,  1,  7,  8, 11,  4,  0])

In [50]:
(f["central"][1:,:]==2).sum(axis=1)

array([102,  37,  93,  48,  32,  25,  43,  19,   6,  41,  31,  40,  16,
         0])